In [ ]:
library(DESeq2)
library(tidyverse)

In [ ]:
#### sampleID file
sampleID = read.table('./source_data/reference/mml_samplelist.txt', header=F, col.names = c("SAMPLE"))

#### condition file
condition = read.table('./source_data/reference/mml_condition.txt', header=F, col.names = c("SAMPLE","AGE","ID","SEX"))

#### GeneCount file
readcount_p <- read.table('./source_data/count_result/GeneCount_mml.txt',header=T)

split_result = str_split(readcount_p$gene_chr,"_", 2)
readcount_p$gene = lapply(X = split_result, FUN = "[", 1)
rownames(readcount_p) <- readcount_p$gene_chr
gene_symbol <- readcount_p$gene

readcount <- readcount_p[,c(2:31)]

In [ ]:
## male
expdesign <- condition[(condition$SEX == 'male'),]
expdesign <- expdesign[order(expdesign$AGE,expdesign$SAMPLE), ]
expdesign$condition = factor(c(rep('G1',15),rep('G2',9),rep('G3',4),rep('G4',2)))
readcount <- readcount[,expdesign$SAMPLE]        
rownames(expdesign) <- expdesign$SAMPLE
expdesign <- expdesign[, c(1,5)]

In [ ]:
############################################
#Time course (LFC = 0 at UP & Down analysis)#
#############################################

dds <- DESeqDataSetFromMatrix(countData = readcount,
                              colData = expdesign,
                              design = ~ condition)

dds_LRT <- DESeq(dds, test="LRT", reduced = ~1)

res_LRT <-results(dds_LRT)


count <- counts(dds_LRT, normalized = TRUE)
design <- as.data.frame(colData(dds_LRT))

write.table(count, file='./source_data/DEmRNA/GeneCount_LRT_normalized_count_TC_mml.txt', sep='\t',col.names = T, row.names = T)